In [1]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
df_w = pd.read_pickle("./muons_165GeV_angle60_delay725ns_GSPS1p2_8Nov_0249.pkl")

In [9]:
df_w.mean()

KeyboardInterrupt: 